Import

In [5]:
import pandas as pd
import numpy as np

# import matplotlib.font_manager

from sklearn import svm

# from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

Data Load

In [6]:
train_df = pd.read_csv('./open/train.csv') # Train
train_df.head()

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,3,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,4,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,6,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,8,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,9,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901


In [7]:
val_df = pd.read_csv('./open/val.csv') # Validation
# val_df.head()

Train/Validation Feature 분포 확인

In [8]:
# train_df.drop(columns=['ID']).hist(bins = 50, figsize = (20,20))
# plt.show()

In [9]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

Validation contamination : [0.0010551491277433877]


In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scale_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V9', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17', 'V18', 'V27']

train_x_scaled = scaler.fit_transform(train_df[scale_cols])

train_x_scaled = pd.DataFrame(train_x_scaled, columns=scale_cols)


Model Define & Fit



In [11]:
# Train dataset은 Label이 존재하지 않음

#Fit the model
nu = 0.3 #val_contamination
kernel = "rbf"
gamma = 0.0001

clf = svm.OneClassSVM(nu = nu, kernel = kernel, gamma = gamma)
clf.fit(train_x_scaled)

OneClassSVM(gamma=0.0001, nu=0.3)

Evaluation : Validation set



In [12]:
def get_pred_label(model_pred):
    # 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [13]:
y_pred_train = clf.predict(train_x_scaled)
n_error_train = y_pred_train[y_pred_train == -1].size
print("n_error_train : ", n_error_train)

n_error_train :  34152


In [14]:
val_y = val_df['Class'] # Label

# scaler_val= StandardScaler()

val_x_scaled = scaler.transform(val_df[scale_cols])
val_x_scaled = pd.DataFrame(val_x_scaled, columns=scale_cols)

val_pred = clf.predict(val_x_scaled) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

Validation F1 Score : [0.416394538409323]
              precision    recall  f1-score   support

           0       1.00      0.70      0.83     28432
           1       0.00      0.97      0.01        30

    accuracy                           0.70     28462
   macro avg       0.50      0.84      0.42     28462
weighted avg       1.00      0.70      0.83     28462



In [ ]:
pd.DataFrame(val_y).describe()

In [ ]:
pd.DataFrame(val_pred).describe()

In [ ]:
n_error_valid = val_pred[val_pred == 1].size
print("n_error_valid : ", n_error_valid)

Inference : Test set



In [ ]:
test_df = pd.read_csv('./open/test.csv') # Train
test_df.head()

In [ ]:
# scaler_test_x = StandardScaler()

test_x_scaled = scaler.transform(test_df[scale_cols])
test_x_scaled = pd.DataFrame(test_x_scaled, columns=scale_cols)


test_pred = clf.predict(test_x_scaled) # model prediction
test_pred = get_pred_label(test_pred)


In [141]:
test_pred.shape

(142503,)

In [148]:
pd.DataFrame(test_pred)[0].unique()

array([0, 1], dtype=int64)

In [144]:
n_error_test = test_pred[test_pred == 1].size
print("n_error_test : ", n_error_test)

n_error_test :  183


Submission


In [ ]:
submit = pd.read_csv('./open/sample_submission.csv')
submit.head()

In [ ]:
submit['Class'] = test_pred
submit.to_csv('./submit.csv', index=False)

파라미터별 validset f1 score

In [139]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30']

train_x_scaled = scaler.fit_transform(train_df[scale_cols])
train_x_scaled = pd.DataFrame(train_x_scaled, columns=scale_cols)


val_y = val_df['Class'] # Label

val_x_scaled = scaler.transform(val_df[scale_cols])
val_x_scaled = pd.DataFrame(val_x_scaled, columns=scale_cols)


#Fit the model

nu = 0.002 # val_contamination
kernel = "rbf"
gamma = 0.0001

clf = svm.OneClassSVM(nu = nu, kernel = kernel, gamma = gamma)
clf.fit(train_x_scaled)

val_pred = clf.predict(val_x_scaled) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(val_score)

0.6772675198335893


In [140]:
val_contamination

0.0010551491277433877